# Filtering rows 3: multiple filter conditions
By the end of this lecture you will be able to:
- use multiple AND conditions in `filter`
- use multiple OR conditions in `filter`
- optimise mulitiple conditions in lazy mode

In [ ]:
import polars as pl

In [ ]:
csvFile = "../data/titanic.csv"

In [ ]:
df = pl.read_csv(csvFile)

## Multiple conditions

### Apply `AND` conditions

We can apply filter `AND` conditions so all conditions must be met by **chaining** calls to `filter`.

In this example we keep all first class passengers that are over 70

In [ ]:
(
    df
    .filter(
        pl.col('Pclass') == 1
    )
    .filter(
        pl.col('Age') > 70
    )
    .head(3)
)

In eager mode chaining requires a pass through the `DataFrame` for each `filter` call.

It is better to **concatenate** multiple `AND` conditions in a single `filter` call using `&`

In [ ]:
(
    df
    .filter(
        (pl.col('Age') > 70) & (pl.col('Pclass') == 1)
    )
    .head(2)
)

### Apply an AND condition using `pl.all_horizontal`
Specifying multiple conditions in chained `filters` or using `&` is fine when we have a small number of conditions to apply. However, we can use the `pl.all_horizontal` method when we want to apply an AND condition on many columns.

In this example we:
- first call `pl.all().is_not_null()` to create a Boolean `DataFrame` where each call is True if the underlying value is not `null`
- then call `pl.all_horizontal` to find rows where all values are `True` (i.e. all values are not `null`)

In [ ]:
(
    df
    .filter(
        pl.all_horizontal(
            pl.all().is_not_null()
        )
    )
    .head(2)
)

### Apply `AND` condition on a range

We use `in_between` to apply a condition on a range. In this case we are looking for values **greater than or equal to** 10 and **less than or equal to** 13

In [ ]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13)
    )
    .head(2)
)

We use the `closed` argument to specify if we want the range to be open, closed on both sides or open on the left or right. The default is for the range to be closed (with a value of `"both"`). 

In this example we are looking for values from 10 to 13 exclusive of the boundaries

In [ ]:
(
    df
    .filter(
        pl.col("Age").is_between(10,13,closed="none")
    )
    .head(2)
)

### Apply `OR` conditions

We can apply an OR filter using the pipe `|` operator.

In this example we look for rows where the passenger is over 70 OR the passenger is in first class

In [ ]:
(
    df
    .filter(
        (pl.col('Age') > 70) | (pl.col('Pclass') == 1)
    )
    .head(2)
)

One kind of OR condition is when we want to check if values are in a `list`. We can do this with `is_in`

In [ ]:
(
    df
    .filter(
        pl.col('Pclass').is_in([2,3])
    )
    .head(3)
)

### Multiple conditions in lazy mode
In *lazy mode* if we pass multiple `filter` calls then the query optimizer combines these into a *single condition* inside `SELECTION`.

In this example we filter for first class passengers over the age of 70.

In [ ]:
df = (
    pl.scan_csv(csvFile)
    .filter(
        pl.col('Pclass')==1
    )
    .filter(
        (pl.col('Age') > 70)
    )
)
print(df.explain())

## Exercises
In the exercises you will develop your understanding of:
- applying multiple AND conditions
- applying multiple OR conditions

### Exercise 1 
Create a `DataFrame` where `Age` is between 30 and 50 (including the lower bound) and the passenger is in 2nd class. Do this in eager mode in a single pass through the `DataFrame`

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

### Exercise 2
Return all the rows of the `DataFrame` where at least one column on the row is `null` (excluding the `Cabin` column with many `null` values)

In [ ]:
(
    pl.read_csv(csvFile)
    .drop("Cabin")
    <blank>
)

### Exercise 3
Create a `DataFrame` where the passengers got on in Cork ("C") or Southampton ("S") using the pipe operator

In [ ]:
(
    pl.read_csv(csvFile)
    <blank>
)

Do this again using the `is_in` approach from the last lecture

## Solutions
### Solution to Exercise 1
Create a `DataFrame` where `Age` is between 30 and 50 (including the lower bound) and the passenger is in 2nd class. Do this in eager mode in a single pass through the `DataFrame`

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(
        (pl.col('Age').is_between(30,50,closed="left")) & (pl.col('Pclass')==2)
    )
)

### Solution to Exercise 2
Return all the rows of the `DataFrame` where at least one column on the row is `null` (excluding the `Cabin` column with many `null` values)

In [ ]:
(
    pl.read_csv(csvFile)
    .drop("Cabin")
    .filter(
        pl.any_horizontal(
            pl.all().is_null()
        )
    )
)

### Solution to Exercise 3
Create a `DataFrame` where the passengers got on in Cork ("C") or Southampton ("S") using the pipe operator

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(
        (pl.col('Embarked') == "C") | (pl.col('Embarked') == "S")
    )
    .head(3)
)

Do this again using the `is_in` approach from the last lecture

In [ ]:
(
    pl.read_csv(csvFile)
    .filter(
        pl.col('Embarked').is_in(["C","S"])
    )
    .head()
)